In [50]:

import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import acorr_breusch_godfrey
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [51]:
df = pd.read_parquet("data/dataset_procesada.parquet", engine="pyarrow")
df

,exportaciones,importaciones,precio_petroleo,igae,pib,consumo_final,inversion_total,tasa_inflacion
fecha,,,,,,,,
2008-10-01,-0.454482,-0.299724,-4.956194,0.037897,0.038064,0.581546,0.414413,-0.060587
2009-01-01,-1.812458,-0.796024,-1.256884,-0.886494,-0.886777,-0.798865,-0.811319,-1.995298
2009-04-01,0.356706,0.603206,1.470491,1.032076,1.032046,0.188213,0.485071,4.119722
2009-07-01,0.512052,0.556132,0.794157,-0.273556,-0.273605,-0.096622,-0.070220,-1.670576
2009-10-01,0.117188,0.069532,0.725736,0.123238,0.123502,0.652331,0.510964,-0.177565
...,...,...,...,...,...,...,...,...
2023-10-01,-0.949204,-0.144119,-0.283497,1.441632,1.441919,1.999727,0.736056,-0.302286
2024-01-01,-1.801562,-1.529920,-0.049535,-1.893434,-1.893470,-1.957061,-1.796246,0.082974
2024-04-01,1.673592,0.960185,0.375641,0.491718,0.491684,-0.176902,1.100735,0.528649


In [52]:
def seleccionar_orden_var(data, maxlags=4):
    """
    Selecciona el orden óptimo del modelo VAR usando criterios de información
    """
    modelo = VAR(data)
    resultados = modelo.select_order(maxlags=maxlags)
    
    print("\n=== CRITERIOS DE SELECCIÓN DE ORDEN ===")
    print(resultados.summary())
    
    # Obtener el orden recomendado por AIC
    orden_aic = resultados.aic
    print(f"\nOrden recomendado por AIC: {orden_aic}")
    
    return orden_aic

In [53]:
def entrenar_var(data, orden):
    """
    Entrena el modelo VAR con el orden especificado
    """
    modelo = VAR(data)
    modelo_fit = modelo.fit(orden)
    
    return modelo_fit

In [54]:
def predecir_var(modelo_fit, steps, data_train):
    """
    Realiza predicciones con el modelo VAR entrenado
    """
    # Obtener los últimos valores para inicializar las predicciones
    lag_order = modelo_fit.k_ar
    forecast_input = data_train.values[-lag_order:]
    
    # Realizar predicción
    forecast = modelo_fit.forecast(forecast_input, steps)
    
    # Convertir a DataFrame con las mismas columnas
    forecast_df = pd.DataFrame(forecast, columns=data_train.columns)
    
    return forecast_df

In [55]:
def evaluar_modelo_var(y_real, y_pred):
    """
    Calcula métricas de evaluación para cada variable del VAR
    """
    resultados = {}
    
    for columna in y_real.columns:
        mae = mean_absolute_error(y_real[columna], y_pred[columna])
        mse = mean_squared_error(y_real[columna], y_pred[columna])
        rmse = np.sqrt(mse)
        
        resultados[columna] = {
            'MAE': mae,
            'MSE': mse,
            'RMSE': rmse
        }
    
    return resultados

In [56]:
def diagnosticos_var(modelo_fit):
    """
    Realiza diagnósticos estadísticos del modelo VAR
    """
    print("\n=== DIAGNÓSTICOS DEL MODELO VAR ===")
    
    # Test de autocorrelación serial (Breusch-Godfrey)
    try:
        test_serial = modelo_fit.test_serial_correlation(lags=10)
        print(f"Test de Autocorrelación Serial (p-value): {test_serial.pvalue:.6f}")
        
        if test_serial.pvalue > 0.05:
            print("No hay evidencia de autocorrelación serial")
        else:
            print("Existe autocorrelación serial en los residuos")
    except:
        print("No se pudo realizar el test de autocorrelación serial")
    
    # Test de normalidad de residuos
    try:
        test_normalidad = modelo_fit.test_normality()
        print(f"Test de Normalidad (p-value): {test_normalidad.pvalue:.6f}")
        
        if test_normalidad.pvalue > 0.05:
            print("Los residuos siguen distribución normal")
        else:
            print("Los residuos NO siguen distribución normal")
    except:
        print("No se pudo realizar el test de normalidad")
    
    print("EVALUACION GENERAL DEL MODELO VAR:")
    print(f"AIC: {modelo_fit.aic:.6f}")
    print(f"BIC: {modelo_fit.bic:.6f}")
    print(f"Log-Likelihood: {modelo_fit.llf:.6f}")
    
    return {
        'AIC': modelo_fit.aic,
        'BIC': modelo_fit.bic,
        'Log_Likelihood': modelo_fit.llf
    }

In [57]:
def test_causalidad_granger(modelo_fit, variable_objetivo='pib'):
    """
    Realiza test de causalidad de Granger
    """
    print(f"\n=== TEST DE CAUSALIDAD DE GRANGER - {variable_objetivo.upper()} ===")
    
    # Test de causalidad para cada variable hacia la variable objetivo
    for var in modelo_fit.names:
        if var != variable_objetivo:
            try:
                resultado = modelo_fit.test_causality(variable_objetivo, [var])
                print(f"{var} -> {variable_objetivo}: p-value = {resultado.pvalue:.6f}")
                
                if resultado.pvalue <= 0.05:
                    print(f"  {var} causa Granger a {variable_objetivo}")
                else:
                    print(f"  {var} NO causa Granger a {variable_objetivo}")
            except:
                print(f"  No se pudo realizar test para {var}")

In [58]:
def analisis_impulso_respuesta(modelo_fit, periodos=10):
    """
    Calcula funciones de impulso-respuesta
    """
    print(f"\n=== ANÁLISIS DE IMPULSO-RESPUESTA ({periodos} períodos) ===")
    
    # Calcular impulso-respuesta
    irf = modelo_fit.irf(periodos)
    
    # Mostrar resumen
    print("Funciones de impulso-respuesta calculadas")
    print(f"Variables: {modelo_fit.names}")
    
    return irf

In [59]:
def generarModelo(df):
    
    # 3. División train/test (80-20)
    n_train = int(len(df) * 0.8)
    
    data_train = df[:n_train]
    data_test = df[n_train:]
    
    print(f"\nDatos de entrenamiento: {data_train.shape}")
    print(f"Datos de prueba: {data_test.shape}")
    
    # 4. Seleccionar orden óptimo del VAR
    print("\n" + "="*50)
    print("SELECCIÓN DE ORDEN ÓPTIMO")
    print("="*50)
    
    orden_optimo = seleccionar_orden_var(data_train)
    
    # 5. Entrenar modelo VAR
    print("\n" + "="*50)
    print("ENTRENAMIENTO DEL MODELO VAR")
    print("="*50)
    
    modelo_var = entrenar_var(data_train, orden_optimo)
    print(f"Modelo VAR({orden_optimo}) entrenado exitosamente")
    
    # 6. Resumen del modelo
    print("\n=== RESUMEN DEL MODELO ===")
    print(modelo_var.summary())
    
    # 7. Realizar predicciones
    print("\n" + "="*50)
    print("REALIZANDO PREDICCIONES")
    print("="*50)
    
    predicciones = predecir_var(modelo_var, len(data_test), data_train)
    print(f"Predicciones realizadas: {predicciones.shape}")
    
    # 8. Evaluar modelo
    print("\n" + "="*50)
    print("EVALUACIÓN DEL MODELO")
    print("="*50)
    
    metricas = evaluar_modelo_var(data_test, predicciones)
    
    for variable, metricas_var in metricas.items():
        print(f"\n--- {variable.upper()} ---")
        for metrica, valor in metricas_var.items():
            print(f"{metrica}: {valor:.6f}")
    
    # 9. Diagnósticos del modelo
    diagnosticos = diagnosticos_var(modelo_var)
    
    # 10. Test de causalidad de Granger
    if 'pib' in df.columns:
        test_causalidad_granger(modelo_var, 'pib')
    
    # 12. Análisis de impulso-respuesta
    irf = analisis_impulso_respuesta(modelo_var)
    
    return modelo_var, predicciones, metricas, irf

*Generacion de modelos*

Se quito MAPE porque no es una métrica adecuada para VAR

In [60]:
generarModelo(df)


Datos de entrenamiento: (52, 8)
Datos de prueba: (13, 8)

SELECCIÓN DE ORDEN ÓPTIMO

=== CRITERIOS DE SELECCIÓN DE ORDEN ===
 VAR Order Selection (* highlights the minimums) 
      AIC         BIC         FPE         HQIC   
-------------------------------------------------
0      -23.89      -23.58   4.212e-11      -23.77
1      -29.04      -26.24   2.523e-13      -27.98
2      -32.71     -27.41*   8.061e-15      -30.70
3      -34.55      -26.76   2.440e-15      -31.61
4     -37.36*      -27.07  7.187e-16*     -33.47*
-------------------------------------------------

Orden recomendado por AIC: 4

ENTRENAMIENTO DEL MODELO VAR
Modelo VAR(4) entrenado exitosamente

=== RESUMEN DEL MODELO ===
  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Wed, 10, Sep, 2025
Time:                     20:50:28
--------------------------------------------------------------------
No. of Equations:         8.00000    BIC:              

(<statsmodels.tsa.vector_ar.var_model.VARResultsWrapper at 0x1a623fc4fa0>,
     exportaciones  importaciones  precio_petroleo      igae       pib  \
 0       -0.965926       0.711337         1.238654  1.894338  1.894833   
 1       -0.798891      -2.259354        -2.972266 -3.357668 -3.358415   
 2       -0.012600       1.359115         0.686123  0.330434  0.330610   
 3        3.763862       0.730523         0.396137  2.499895  2.499891   
 4       -1.617741      -1.381373         0.932842 -0.592921 -0.592467   
 5       -0.324585       1.196974         0.866797 -1.497195 -1.497711   
 6       -0.024640      -0.744607        -2.384626 -0.487391 -0.487489   
 7        0.120619       0.282306         1.317986  1.797441  1.798018   
 8        0.472464       0.354317        -0.026384  1.050954  1.050850   
 9       -0.523155      -1.197320        -0.017682 -3.091587 -3.092125   
 10       1.046008       1.466066         0.433463  1.529854  1.530247   
 11       0.317059      -1.719087    

In [61]:
df2 = df.drop('igae', axis=1)
generarModelo(df2)


Datos de entrenamiento: (52, 7)
Datos de prueba: (13, 7)

SELECCIÓN DE ORDEN ÓPTIMO

=== CRITERIOS DE SELECCIÓN DE ORDEN ===
 VAR Order Selection (* highlights the minimums) 
      AIC         BIC         FPE         HQIC   
-------------------------------------------------
0      -6.455      -6.183    0.001572      -6.352
1      -11.51      -9.331   1.021e-05      -10.69
2      -15.45     -11.36*   2.258e-07      -13.91
3      -16.96      -10.95   7.248e-08      -14.69
4     -18.97*      -11.06  2.196e-08*     -15.98*
-------------------------------------------------

Orden recomendado por AIC: 4

ENTRENAMIENTO DEL MODELO VAR
Modelo VAR(4) entrenado exitosamente

=== RESUMEN DEL MODELO ===
  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Wed, 10, Sep, 2025
Time:                     20:50:28
--------------------------------------------------------------------
No. of Equations:         7.00000    BIC:              

(<statsmodels.tsa.vector_ar.var_model.VARResultsWrapper at 0x1a623fc4f40>,
     exportaciones  importaciones  precio_petroleo       pib  consumo_final  \
 0       -1.491036       0.519349        -0.557015  1.360147       1.784549   
 1       -1.203519      -2.125093        -1.532146 -3.102483      -2.358394   
 2        0.028661       1.620088         1.153354  0.816053       0.037939   
 3        3.344119       1.300784         0.442797  1.802848       1.012721   
 4       -0.485626      -1.636957         0.291739  0.278700       1.164356   
 5       -0.174104       0.053960         0.528555 -1.845360      -1.871375   
 6       -0.248715      -0.375189        -0.422294 -0.226643      -0.383607   
 7       -0.417224       1.128624         0.063236  0.750167       0.518924   
 8        0.153103       0.098032        -0.979006  1.107464       1.590209   
 9       -0.457023      -1.824099         0.602081 -2.228112      -2.054815   
 10       1.402570       2.097790         0.633011  1.34

In [62]:
df3 = df.drop(['igae', 'exportaciones', 'inversion_total'], axis=1)
generarModelo(df3)


Datos de entrenamiento: (52, 5)
Datos de prueba: (13, 5)

SELECCIÓN DE ORDEN ÓPTIMO

=== CRITERIOS DE SELECCIÓN DE ORDEN ===
 VAR Order Selection (* highlights the minimums) 
      AIC         BIC         FPE         HQIC   
-------------------------------------------------
0      -3.455      -3.260     0.03159      -3.381
1      -5.829      -4.660    0.002960      -5.387
2      -7.175      -5.031   0.0007975      -6.365
3      -10.56     -7.444*   2.952e-05      -9.384
4     -11.22*      -7.130  1.833e-05*     -9.676*
-------------------------------------------------

Orden recomendado por AIC: 4

ENTRENAMIENTO DEL MODELO VAR
Modelo VAR(4) entrenado exitosamente

=== RESUMEN DEL MODELO ===
  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Wed, 10, Sep, 2025
Time:                     20:50:28
--------------------------------------------------------------------
No. of Equations:         5.00000    BIC:              

(<statsmodels.tsa.vector_ar.var_model.VARResultsWrapper at 0x1a624022950>,
     importaciones  precio_petroleo       pib  consumo_final  tasa_inflacion
 0        1.294431         0.010487  1.482300       2.006670        0.093691
 1       -2.361905        -0.190426 -2.009676      -1.897864       -0.953358
 2        0.552533        -0.148047  0.057237      -0.455094        1.304594
 3        1.242086         0.129418  0.739904       0.399882       -0.822826
 4       -0.273915        -0.317661  1.131152       1.911180        0.457644
 5       -1.173781         0.128414 -2.239427      -2.064738       -0.880069
 6        0.308051        -0.063481  0.548460      -0.245845        0.985408
 7        0.908813         0.084029  0.372398       0.227682       -0.317401
 8        0.642653         0.040844  1.552251       2.199913        0.051736
 9       -2.409442         0.041462 -2.611033      -2.312123       -0.885260
 10       1.205675         0.105042  0.634761      -0.223712        1.306496
 